In [3]:
#essential libraries

from google_drive import gdrive

from matplotlib.pyplot import show
import numpy as np
import pandas as pd
from scipy.cluster.vq import *
import operator
import matplotlib
reload(matplotlib)
matplotlib.use('Agg')
from matplotlib import pyplot as plt
import pickle
import shelve
import re
from collections import Counter, defaultdict, OrderedDict, deque
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
import scipy
from sklearn.feature_extraction import DictVectorizer
from matplotlib.backends.backend_pdf import PdfPages
import csv
import sys
import math
from copy import deepcopy, copy
import random
import rosetta
from datetime import datetime, timedelta
from operator import itemgetter
from itertools import chain
import os
import gc
import linecache
import multiprocessing
from functools import partial
from itertools import repeat

import pprint
pp = pprint.PrettyPrinter()

import rdflib
from rdflib.namespace import OWL, RDF, RDFS
from rdflib import URIRef

from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import OneClassSVM, SVC
from sklearn.mixture import GMM
from sklearn.mixture import DPGMM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as hier
from scipy.cluster.hierarchy import fcluster
from scipy import spatial
from sklearn.multiclass import OneVsRestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as hier
from scipy import stats

from scipy.spatial.distance import cosine as cosine_similarity

from divergence import gau_js as js_divergence
import building_tokenizer as toker
import brick_parser
reload(brick_parser)
from brick_parser import tagList, tagsetList, equipTagsetList, pointTagsetList, locationTagsetList,\
equalDict, pointTagList, equipTagList, locationTagList, equipPointDict
subTagListDict = dict([('point', pointTagList),
                          ('equip', equipTagList),
                          ('location', locationTagList)
                         ])
subTagsetListDict = dict([('point', pointTagsetList),
                          ('equip', equipTagsetList),
                          ('location', locationTagsetList)
                         ])

#from cmu_parser import cmu_building_parse

debugFlag = False

Init Graph
Load Brick.ttl
Load BrickFrame.ttl


In [4]:
workerNum = 7
coverageNum = 100

In [5]:
def save_fig(fig, name, dpi=400):
	pp = PdfPages(name)
	pp.savefig(fig, bbox_inches='tight', pad_inches=0, dpi=dpi)
	pp.close()

In [6]:
def pick_dict_with_best_n_from_dict(srcDict, n):
    idxCnt = 0
    sortedValueList = sorted(srcDict.values(), reverse=True)[0:n]
    chosenItemList = list()
    
    for score in sortedValueList:
        for key, val in srcDict.items():
            if idxCnt>=n:
                break
            if val==score:
                chosenItemList.append((key,val))
                idxCnt += 1
        if idxCnt>=n:
                break
    return OrderedDict(chosenItemList)

In [7]:
#from __future__ import print_function
from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o))#, file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)

In [8]:
def get_equip_points(equip, g):
    pointList = list()
    hasPointRef = URIRef(uriPrefix+'hasPoint')
    if equip=='vav':
        equipRef = URIRef(uriPrefix+'VAV')
    elif equip=='ahu':
        equipRef = URIRef(uriPrefix+'AHU')
    elif equip=='vfd':
        equipRef = URIRef(uriPrefix+'VFD')
    elif equip=='chilled_water_pump':
        equipRef = URIRef(uriPrefix+'Chilled_Water_Pump')
    elif equip=='hot_water_pump':
        equipRef = URIRef(uriPrefix+'Hot_Water_Pump')
    elif equip=='crac':
        equipRef = URIRef(uriPrefix+'CRAC')
    elif equip=='exhaust_fan':
        equipRef = URIRef(uriPrefix+'AHU')
    elif equip=='return_fan':
        equipRef = URIRef(uriPrefix+'AHU')
    elif equip=='supply_fan':
        equipRef = URIRef(uriPrefix+'AHU')
    #elif equip=='return_fan':
    #    equipRef = URIRef(uriPrefix+'Return_Fan')
    #elif equip=='exhaust_fan':
    #    equipRef = URIRef(uriPrefix+'Exhaust_Fan')
    #elif equip=='supply_fan':
    #    equipRef = URIRef(uriPrefix+'Supply_Fan')
    else:
        print ("equip name %s is not in the schema" % equip)
        assert(False)
    for superClass in g.objects(equipRef,RDFS.subClassOf):
        #print triple
        #for tri in g.triples((triple, None, None)):
        if (superClass, OWL.onProperty, hasPointRef) in g:
            for point in g.objects(superClass, OWL.someValuesFrom):
                pointList.append(point.decode().split("#")[1].lower().split('_'))
    return pointList

def equip_ref_to_type(equipRef):
    if equipRef==None:
        return None
    equipRefList = equipRef.split('_')
    equipTypeList = list()
    for word in equipRefList:
        numList = re.findall('\d+', word)
        if len(numList)==0:
            equipTypeList.append(word)
    return str('_'.join(equipTypeList))

def equip_ref_to_ref(equipRef):
    if equipRef==None:
        return None
    equipRefList = equipRef.split('_')
    equipTypeList = list()
    for word in equipRefList:
        numList = re.findall('\d+', word)
        if len(numList)!=0:
            return word
    return '9999'

def get_alpha_word(word):
    alphaWordList = re.findall('[a-zA-Z]+', word)
    if len(alphaWordList)>0:
        alphaWord = alphaWordList[0]
        return alphaWord
    else:
        return ''

def get_num_word(word):
    wordList = re.findall('[b]?\d+[abc]?\_?\d*', word)
    if len(wordList)>0:
        foundWord = wordList[0]
        return foundWord
    else:
        return '9999'
    

In [9]:
class SentenceTagsetsCombination:
    weightDictHistory = None
    sentence = None
    
    def determine_word_belongs_to_sentence(self, word, tagList):
        #closestTagList = scoreDictDict[word].keys()[0].split()
        #return sum([1.0 if tag in tagList else 0.0 for tag in closestTagList]) / float(len(closestTagList)) >=0.2
        subScoreDict = pick_dict_with_best_n_from_dict(scoreDictDict[word], 3)
        totalScore = 0
        for tagset, score in subScoreDict.items():
            for tag in tagset.split():
                if tag in tagList:
                    totalScore += score / len(tagList)
        totalScore /= sum(subScoreDict.values())
        return score>=0.2
    
    def __init__(self, sentence, subTagListDict):
        self.scoreHistory = [0]
        self.sentence = sentence
        wordScoreDictDict = dict()
        for tagsetType in subTagListDict.keys()+['dummy']:
            wordScoreDictDict[tagsetType] = dict()
        
        for word in sentence:
            if len(word)<=1 or '' in scoreDictDict[word].keys():
                wordScoreDictDict['dummy'][word] = 1.0
                for tagsetType in subTagListDict.keys():
                    wordScoreDictDict[tagsetType][word] = 0.0
                continue
            
            wordOccurrenceDict = dict()
            for tagsetType, tagList in subTagListDict.items():
                wordOccurrenceDict[tagsetType] = 1.0 if self.determine_word_belongs_to_sentence(word, tagList) else 0.0
            totalOcc = sum(wordOccurrenceDict.values())
            if totalOcc==0:
                wordScoreDictDict['dummy'][word] = 1.0
                for tagsetType in wordOccurrenceDict.keys():
                    wordScoreDictDict[tagsetType][word] = 0.0
            else:
                wordScoreDictDict['dummy'][word]= 0.0
                for tagsetType, wordOccurrence in wordOccurrenceDict.items():
                    wordScoreDictDict[tagsetType][word] = wordOccurrence / totalOcc
        
        self.weightDictHistory = [wordScoreDictDict]
        
    def set_sentence(self, sentence):
        self.sentence = sentence
        
    def get_sentence(self):
        return self.sentence
    
    def get_weight_dict(self, tagsetType):
        return self.weightDictHistory[-1][tagsetType]
    
    def get_total_weight_dict(self):
        return self.weightDictHistory[-1]
    
    def update_total_weight_dict(self, newWeightDict):
        aWeightDict = self.weightDictHistory[-1]['dummy']
        assert(len(newWeightDict.keys())==len(self.weightDictHistory[-1].keys()))
        for word in aWeightDict.keys():
            #try:
            assert(np.allclose(1.0, sum([weightDict[word] for weightDict in newWeightDict.values()])))
            #except:
            #    print "weightDict wrong: "
            #    print "word:", word
            #    print "original weightDict: ", newWeightDict
            #    print "Summation of the word's weights", sum([weightDict[word] for weightDict in newWeightDict.values()])
            #    assert(False)
        #self.weightDictHistory.append(newWeightDict)
        self.weightDictHistory[0] = newWeightDict
        return self

In [10]:
class SubSentenceTagsetPair:
    localScoreDict = dict() # similar to scoreDictDict but only contains words existing in the sentence
    sentence = list()
    targetTagset = ''
    origSentence = list()
    predLable = ''
    score = 0 
    direction = ''
    scoreHistory = []
    
    def __init__(self, origSentence, localScoreDict, targetTagset, score=0):
        self.origSentence = origSentence
        self.sentence = origSentence
        self.localScoreDict = localScoreDict
        self.score = score
        self.scoreHistory = [score]
        self.targetTagset = targetTagset
    
    def append_score_history(self, score):
        self.scoreHistory.append(score)
    def get_score_history(self):
        return self.scoreHistory
    def set_score_dict(self, localScoreDict):
        self.localScoreDict = localScoreDict
    def set_direction(self, flag):
        if not (flag=='+' or flag=='-'):
            print 'wrong direction flag'
            assert(False)
        else:
            self.direction = flag
    def get_direction(self):
        return self.direction
    def get_score_dict(self):
        return self.localScoreDict
    def set_sentence(self, sentence):
        self.sentence = sentence
    def get_sentence(self):
        return self.sentence
    def set_score(self, score):
        self.score = score
    def get_score(self):
        return self.score
    def get_target_tagset(self):
        return self.targetTagset

In [ ]:
import building_tokenizer
reload(building_tokenizer)
import building_tokenizer as toker

In [ ]:
buildingName = 'ebu3b'
notUcsdBuildings = ['ghc']

tokenType = 'NoNumber'

if not buildingName in notUcsdBuildings:
    naeDict = dict()
    naeDict['bonner'] = ["607", "608", "609", "557", "610"]
    naeDict['ap_m'] = ['514', '513','604']
    naeDict['bsb'] = ['519', '568', '567', '566', '564', '565']
    naeDict['ebu3b'] = ["505", "506"]
    naeList = naeDict[buildingName]

    labeledFile = 'metadata/' + buildingName + '_sensor_types_location.csv'
    with open(labeledFile, 'rb') as fp:
        #truthDF = pd.read_excel(fp)
        truthDF = pd.DataFrame.from_csv(fp)
        #truthDF = truthDF.set_index(keys='Unique Identifier')

    wordFeatFile = 'data/wordfeat_'+buildingName+'.pkl'

    tokenTypeList = ['NoNumber', 'Alphanumeric', 'AlphaAndNum', 'NumAsSingleWord']

    bacnetTypeMapDF = pd.DataFrame.from_csv('metadata/bacnettype_mapping.csv')
    unitMap = pd.read_csv('metadata/unit_mapping.csv').set_index('unit')
    for val in Counter(unitMap.keys()).values():
        if val>1:
            "Unit map file ERROR"
            assert(False)
            
            
    trueDF = pd.DataFrame.from_csv('metadata/'+buildingName+'_sensor_types_location.csv')
    sensorDF, nameList, jcinameList, descList, unitList, bacnettypeList, wordList = \
    toker.structure_metadata(buildingName=buildingName, tokenType=tokenType, \
                             validSrcidList=trueDF.index.tolist(), withDotFlag=False)

    origSensorDF = deepcopy(sensorDF)
    origNameList = deepcopy(nameList)
    origJcinameList = deepcopy(jcinameList)
    origDescList = deepcopy(descList)
    origUnitList = deepcopy(unitList)
    origBacnettypeList = deepcopy(bacnettypeList)
    origWordList = deepcopy(wordList)

    with open('data/'+buildingName+'_str_score_dict.pkl', 'rb') as fp:
        scoreDictDict = pickle.load(fp)

    ### If a word is exactly matched with another, fix it.
    for word, scoreDict in scoreDictDict.items():
        if word in scoreDict.keys():
            scoreDictDict[word] = {word:1}
    
    _, rawNameList, rawJcinameList, rawDescList, _, _, _ = toker.structure_metadata(buildingName=buildingName, tokenType='AlphaAndNum', \
                             validSrcidList=trueDF.index.tolist(), withDotFlag=False)
    
    
else: 
    filename = 'metadata/'+buildingName+'_sensor_types_location.csv'
    #   filename = 'metadata/%s_sensor_types_location.csv'%buildingName
    df = pd.read_csv(filename)
    sentenceList = list()
    
    
    for raw in df['bas_raw'].tolist():
        sentenceList.append(toker.tokenize(tokenType, raw))

In [ ]:
#Select useful indices only
'''
srcidList = sensorDF.index.to_series().tolist()
idxList = list()
addedTypeList = list()
cnt = 0
for srcid, row in trueDF.iterrows():
    pointType = row['Schema Label']
    if pointType not in addedTypeList:
        addedTypeList.append(pointType)
        i = srcidList.index(srcid)
        idxList.append(i)
        cnt +=1 
        if cnt>=100:
            break
'''
idxList = range(0,coverageNum)

In [ ]:
listIndexFunc = lambda l, i: [l[ii] for ii in i]
sensorDF = origSensorDF.iloc[idxList]
nameList = listIndexFunc(origNameList, idxList)
jcinameList = listIndexFunc(origJcinameList, idxList)
descList = listIndexFunc(origDescList, idxList)
unitList = listIndexFunc(origUnitList, idxList)
bacnettypeList = listIndexFunc(origBacnettypeList, idxList)


In [ ]:
len(nameList)

In [ ]:
def calc_max_score_dict(scoreDictDict):
    maxScoreDict = dict()
    for word, scoreDict in scoreDictDict.items():
        maxScoreDict[word] = max(scoreDict.values())
    return maxScoreDict


In [ ]:
### Human Input
#scoreDictDict['ebu'] = {'building':1}

scoreDictDict['ebu'] = {'':1}
scoreDictDict['vma'] = {'':1}
scoreDictDict['nae'] = {'':1}
scoreDictDict['meter'] = {'meter': 1}
#scoreDictDict['ap'] = {'':1}
#scoreDictDict['trunk'] = {'':1}

scoreDictDict['rm'] = {'room':1}
#scoreDictDict['actual'] = {'effective':1}
#scoreDictDict['percent'] = {'':1}

maxScoreDict = calc_max_score_dict(scoreDictDict)

origScoreDictDict = deepcopy(scoreDictDict)

In [ ]:
def manual_def(word, trueTagset):
    scoreDict = scoreDictDict[word]
    scoreDictDict[word] = dict((tagset,1) if turTagset==tagset else (tagset,0) for tagset in scoreDict.keys())

In [ ]:
### Initialization sentence separation
#### We need equip sentence
### We need type sentence, equip sentence, location sentence

# Init no meaning sentence
sentenceList = list()
adder = lambda a,b:a+b
splitter = lambda s:s.split()
sentenceList = [reduce(adder, map(splitter, dataList))\
                #for dataList in zip(nameList, jcinameList, descList, unitList, bacnettypeList)]
                for dataList in zip(nameList, jcinameList, unitList, bacnettypeList)]
                #for dataList in zip(nameList, jcinameList, unitList)]
                #for dataList in zip(nameList, jcinameList)]
rawSentenceList = [reduce(adder, map(splitter, dataList))\
                #for dataList in zip(rawNameList, rawJcinameList, rawDescList, unitList, bacnettypeList)]
                for dataList in zip(rawNameList, rawJcinameList, unitList, bacnettypeList)]
                #for dataList in zip(rawNameList, rawJcinameList, unitList)]
                #for dataList in zip(rawNameList, rawJcinameList)]

# Make type, equip, location sentences
equipSentenceList = list()
pointSentenceList = list()
locationSentenceList = list()

for sentence in sentenceList:
    equipSentence = list()
    pointSentence = list()
    locationSentence = list()
    for word in sentence:
        closestTagList = scoreDictDict[word].keys()[0].split()
        if len(closestTagList)==0:
            continue
        if sum([1 if tag in equipTagList else 0 for tag in closestTagList]) / float(len(closestTagList)) >=0.3:
            equipSentence.append(word)
        if sum([1 if tag in pointTagList else 0 for tag in closestTagList]) / float(len(closestTagList)) >=0.3:
            pointSentence.append(word)
        if sum([1 if tag in locationTagList else 0 for tag in closestTagList]) / float(len(closestTagList)) >=0.3:
            locationSentence.append(word)
    equipSentenceList.append(equipSentence)
    pointSentenceList.append(pointSentence)
    locationSentenceList.append(locationSentence)

#assert(len(trueDF)==len(pointSentenceList))

In [ ]:
def calc_tfidf_sub(cntDict, idfDict):
    tfidfList = list()
    maxCnt = max(cntDict.values())
    for tag, idf in idfDict.items():
        if not tag in cntDict.keys():
            tfidf = 0
        else:
            tfidf = (0.5 + 0.5 * cntDict[tag] / maxCnt)* idf
        tfidfList.append(tfidf)
    return np.asarray(tfidfList)

def sentence_tagset_score_segmented_tfidf(sentence, targetTagset):
    targetTagset = targetTagset.split('_')
    compDataLen = 5
    sentenceCntDict = defaultdict(float)
    
    for word in sentence:
        totalWordScore = 1.0/len(sentence)
        scoreDict = dict(scoreDictDict[word].items()[0:compDataLen])
        localScoreSum = float(sum(scoreDict.values())) #* len(sentence)
        for candTagset, score in scoreDict.items():
            candTagList = candTagset.split()
            candTagScore = score / localScoreSum / len(candTagList)
            for candTag in candTagList:
                sentenceCntDict[candTag] += candTagScore
                
    tagsetCntDict = defaultdict(float)
    #avgCnt = np.mean(sentenceCntDict.values())
    for tag in targetTagset:
        tagsetCntDict[tag] += 1.0/len(targetTagset)
        
    entireCntDict = OrderedDict(sentenceCntDict)
    for tag in targetTagset:
        if tag not in sentenceCntDict.keys():
            entireCntDict[tag] = tagsetCntDict[tag]
    
    entireCntDict = OrderedDict(entireCntDict)
    totalCnt = sum(entireCntDict.values())
    idfValues = [np.log(totalCnt/cnt) for cnt in entireCntDict.values()]
    idfDict = OrderedDict()
    for tag, idf in zip(entireCntDict.keys(), idfValues):
        idfDict[tag]  = idf
    
    sentenceTfidf = calc_tfidf_sub(sentenceCntDict, idfDict)
    tagsetTfidf = calc_tfidf_sub(tagsetCntDict, idfDict)
    return 1 - cosine_similarity(sentenceTfidf, tagsetTfidf)

#sentence_tagset_score = sentence_tagset_score_segmented_tfidf

In [ ]:
# Maybe need log-scale compensation of redundant tags or words

needDataLen =12

def sentence_tagset_score_word_and_sentence_together(sentence, targetTagset, scoreDictDict=scoreDictDict):
    origSentence = sentence
    if type(targetTagset)==str or type(targetTagset)==unicode:
        targetTagset = targetTagset.split('_')
    
    totalScore = 0 
    notUsedWordCntDict = dict()
    for word in sentence:
        notUsedWordCntDict[word] = 0
        
    tagsetCntDict = dict()
    for tag in targetTagset:
        tagsetCntDict[tag] = 0
    
    for word in sentence:
        scoreDict = dict(scoreDictDict[word].items()[0:needDataLen])
        localScoreSum = float(sum(scoreDict.values()))
        for candTagset, score in scoreDict.items():
            candTags = candTagset.split()
            for candTag in candTags:
                candTagScore = score / localScoreSum / len(candTags)
                if candTag in targetTagset:
                    tagsetCntDict[candTag] += candTagScore
                else:
                    notUsedWordCntDict[word] += candTagScore
    tagsetScoreDict = dict()
    for word, cnt in tagsetCntDict.items():
        tagsetScoreDict[word] = np.log(1+cnt)
    return (len(sentence)-sum(notUsedWordCntDict.values()))/len(sentence) * sum(tagsetScoreDict.values()) / len(targetTagset)
    
sentence_tagset_score = sentence_tagset_score_word_and_sentence_together

In [ ]:
def sentence_tagset_score_with_weight_inside(sentence, targetTagset, weightDict, scoreDictDict=scoreDictDict, \
                                             maxScoreDict=maxScoreDict, totalScore=0, sentenceLen=None):
    if type(targetTagset)==str or type(targetTagset)==unicode:
        targetTagset = targetTagset.split('_')
    
    #TODO: Choose between those two.
    if sentenceLen==None:
        sentenceLen = len(sentence)
    
    totalScoreForTagset = 0
    for word in sentence:
        totalScoreForTagset += weightDict[word]
    
    if totalScoreForTagset==0:
        return 0, 0, 0
    
    for word in deepcopy(sentence):
        if len(word)<=1:
            sentence.remove(word)
    
    notUsedWordCntDict = dict()
    for word in sentence:
        notUsedWordCntDict[word] = 0
        
    tagsetCntDict = dict()
    for tag in targetTagset:
        tagsetCntDict[tag] = 0
        
    sysWordUsageDict = defaultdict(float)
    
    for word in sentence:
        wordScore = weightDict[word] * max(scoreDictDict[word].values()) / maxScoreDict[word]
        
        if wordScore==0:
            continue
            
        scoreDict = dict(scoreDictDict[word].items()[0:needDataLen])
        localScoreSum = float(sum(scoreDict.values()))
        for candTagset, score in scoreDict.items():
            candTags = candTagset.split()
            for candTag in candTags:
                candTagScore = score / localScoreSum / len(candTags) * wordScore
                if candTag in targetTagset:
                    tagsetCntDict[candTag] += candTagScore
                else:
                    notUsedWordCntDict[word] += candTagScore
    tagsetScoreDict = dict()
    for tag, cnt in tagsetCntDict.items():
        tagsetScoreDict[tag] = np.log(1+cnt)
    #for tag, cnt in tagsetCntDict.items():
    #    tagsetScoreDict[tag] = cnt / (totalScoreForTagset/float(len(targetTagset)))
    
    
    notUsedWordScoreDict = dict()
    for word, cnt in notUsedWordCntDict.items():
        notUsedWordScoreDict[word] = np.log(1+cnt)
    #print "notusedscore: ", notUsedWordScoreDict['chwp']
        
    
    
    sentenceUsage = sum(tagsetCntDict.values()) / float(sentenceLen)
    tagsetCoverage = sum(tagsetScoreDict.values()) / (len(targetTagset)*np.log(1+totalScoreForTagset/len(targetTagset)))
    #tagsetCoverage = sum(tagsetCntDict.values()) / totalScoreForTagset
    candScore = sentenceUsage  * tagsetCoverage
    
    #print "---"
    #print sentenceUsage
    #print tagsetCoverage
    #pp.pprint(tagsetScoreDict)
    #print "---"
    
    return candScore, sentenceUsage, tagsetCoverage#, notUsedWordCntDict
    
    
def sentence_tagset_score_with_weight(sentence, targetTagset, weightDict, scoreDictDict=scoreDictDict, maxScoreDict=maxScoreDict, totalScore=0):
    candScore, _, _ = sentence_tagset_score_with_weight_inside(sentence, targetTagset, weightDict, scoreDictDict=scoreDictDict, maxScoreDict=maxScoreDict, totalScore=0)
    return candScore

In [ ]:
def calc_score_altogether_in_sentence(comb, candDict, scoreDictDict):
    sentence = comb.get_sentence()
    #TODO: Check which is good between below two lines
    #sentenceTotalScore = len(sentence) - sum(comb.get_weight_dict('dummy').values())
    sentenceTotalScore = len(sentence)
    sentenceUsageSum = 0
    tagsetCoverageList = list()
    weightSumList = list()
    candScoreSum = 0
    #TODO: Choose to include this or not
    #sentenceLen = len(sentence) - sum(comb.get_weight_dict('dummy').values())
    sentenceLen = None
    
    for tagsetType, cand in candDict.items():
        # TODO: Return to the original
        candScore, sentenceUsage, tagsetCoverage = sentence_tagset_score_with_weight_inside(sentence, cand, \
                                        comb.get_weight_dict(tagsetType), scoreDictDict, sentenceLen=sentenceLen)
        sentenceUsageSum += sentenceUsage
        tagsetCoverageList.append(tagsetCoverage)
        weightSumList.append(sum(comb.get_weight_dict(tagsetType).values())) 
        candScoreSum += candScore
    sentenceUsage = sentenceUsageSum / sentenceTotalScore
    totalTagsetCoverage = sum([cov*weight for cov, weight in zip(tagsetCoverageList, weightSumList)])/float(sum(weightSumList))
    
    #print "sentence usage: ", sentenceUsage
    #print "tagset coverage: ", totalTagsetCoverage
    #return totalTagsetCoverage * sentenceUsage
    return candScoreSum

In [ ]:
extractSubDict = lambda origDict, tagsetTypeList: \
            dict((tagsetType, subTagList) for tagsetType, subTagList in origDict.items() if tagsetType in tagsetTypeList)
subTagListDictList = [extractSubDict(subTagListDict, ['point']),\
                      extractSubDict(subTagListDict, ['point', 'equip']),\
                      extractSubDict(subTagListDict, ['point', 'equip', 'location']),\
                      extractSubDict(subTagListDict, ['point', 'location'])
                      ]
                      
combCandDict = defaultdict(list)
scoreDictDict = dict()

for sentence in sentenceList[0:coverageNum]:
#for sentence in sentenceList[2:3]:
    for word in sentence:
        if word not in scoreDictDict.keys():
            scoreDictDict[word] = origScoreDictDict[word]

            

In [ ]:
def init_comb((sentence, subTagListDictList, scoreDictDict, reduceCandNum)):
    combCand = list()
    for tagListDict in subTagListDictList:
        tagTypeList = tagListDict.keys()
        comb = SentenceTagsetsCombination(sentence, tagListDict)
        sentenceTotalScore = len(comb.get_sentence()) - sum(comb.get_weight_dict('dummy').values())
        weightDictDict = comb.get_total_weight_dict()
        candDictList = list()
        multipleTagsetScoreDict = defaultdict(dict)
        for tagsetType, weightDict in weightDictDict.items():
            if tagsetType=='dummy':
                continue
            tagsetList = subTagsetListDict[tagsetType]
            for tagset in tagsetList:
                #multipleTagsetScoreDict[tagsetType][tagset] = sentence_tagset_score_with_weight(sentence, \
#                                                                                                tagset, weightDict)
                multipleTagsetScoreDict[tagsetType][tagset] = calc_score_altogether_in_sentence(comb, {tagsetType:tagset}, \
                                                                                                scoreDictDict)
        multipleTagsetScoreDict = dict(multipleTagsetScoreDict)
        
        pointItemList = list()
        equipPointItemList = list()
        
        pointEquipCandNum = 5 - reduceCandNum
        if pointEquipCandNum<1:
            pointEquipCandNum = 1
        locationCandNum = 3 - reduceCandNum
        if locationCandNum <2:
            locationcandNum = 2
        
        chosenPointTagsetDict = pick_dict_with_best_n_from_dict(multipleTagsetScoreDict['point'], 5 - reduceCandNum)
        #chosenItemList 
        pointItemList += [({'point':pointTagset}, score) for pointTagset, score in chosenPointTagsetDict.items()]
        
        equipPointItemList = list()
        
        if 'equip' in tagTypeList:
            chosenEquipTagsetDict = pick_dict_with_best_n_from_dict(multipleTagsetScoreDict['equip'], 5 - reduceCandNum)
            for equipTagset, equipScore in chosenEquipTagsetDict.items():
                for pointTagset, pointScore in chosenPointTagsetDict.items():
                    if pointTagset in equipPointDict[equipTagset]:
                        candDict = {'point': pointTagset, 'equip':equipTagset}
                        equipPointItemList.append((candDict, calc_score_altogether_in_sentence(comb, candDict, scoreDictDict)))
        '''
        if 'equip' in tagTypeList:
            chosenEquipTagsetDict = pick_dict_with_best_n_from_dict(multipleTagsetScoreDict['equip'], 4 - reduceCandNum)
            for equipTagset, equipScore in chosenEquipTagsetDict.items():
                for pointTagset, pointScore in chosenPointTagsetDict.items():
                    candDict = {'point': pointTagset, 'equip':equipTagset}
                    equipPointItemList.append((candDict, calc_score_altogether_in_sentence(comb, candDict, scoreDictDict)))
        '''
        
        if 'location' in tagTypeList:
            chosenLocationTagsetDict = pick_dict_with_best_n_from_dict(multipleTagsetScoreDict['location'], 3 - reduceCandNum)
            locationEquipPointItemList = list()
            locationPointItemList = list()
            for locationTagset, locationScore in chosenLocationTagsetDict.items():
                if 'equip' in tagTypeList:
                    for item in equipPointItemList:
                        item[0]['location'] = locationTagset
                        locationEquipPointItemList.append((deepcopy(item[0]), item[1]+locationScore))
                else:
                    for item in pointItemList:
                        item[0]['location'] = locationTagset
                        locationPointItemList.append((deepcopy(item[0]), item[1]+locationScore))
                    
                    
        if 'equip' in tagTypeList:
            if 'location' in tagTypeList:
                chosenItemList = locationEquipPointItemList
            else:
                chosenItemList = equipPointItemList
        elif 'location' in tagTypeList:
            chosenItemList = locationPointItemList
        else:
            chosenItemList = pointItemList
        combCand.append((comb,chosenItemList))
    return combCand

mpFlag = True
beginTime =  datetime.now()
if mpFlag:
    pool = multiprocessing.Pool(workerNum)
    combTupleList = pool.map(init_comb, zip(sentenceList, repeat(subTagListDictList), repeat(scoreDictDict), repeat(0)))
    pool.terminate()
    pool.close()
    pool.join()
else:
    combTupleList = map(init_comb, zip(sentenceList, repeat(subTagListDictList), repeat(scoreDictDict), repeat(0)))
    pass
combCandDict = dict((i,combTuple) for i, combTuple in enumerate(combTupleList))
endTime = datetime.now()

origCombCandDict = deepcopy(combCandDict)

print (endTime-beginTime).total_seconds()/60, 'minutes'

In [24]:
def update_comb_cand_dict_score(combCandDict, scoreDictDict=scoreDictDict):
    for idx, weightTypeList in combCandDict.items():
        typeNum = len(weightTypeList)
        for i, (comb, candList) in enumerate(weightTypeList):
            localCandScoreSum = sum([candScore for _,candScore in candList])
            #TODO: Validate this
            for j, (candDict, _) in enumerate(candList):
                '''
                sentenceUsageSum = 0
                tagsetCoverageList = list()
                weightSumList = list()
                for tagsetType, cand in candDict.items():
                    candScore, sentenceUsage, tagsetCoverage = sentence_tagset_score_with_weight_inside(comb.get_sentence(), \
                                                                     cand, comb.get_weight_dict(tagsetType), scoreDictDict)
                    sentenceUsageSum += sentenceUsage
                    tagsetCoverageList.append(tagsetCoverage)
                    weightSumList.append(sum(comb.get_weight_dict(tagsetType).values())) 
                sentenceUsage = sentenceUsageSum / sentenceTotalScore
                totalTagsetCoverage = sum([cov*weight for cov, weight in zip(tagsetCoverageList, weightSumList)])/float(sum(weightSumList))
                '''
                #candList[j] = (candDict, sentenceUsage * totalTagsetCoverage)
                candList[j] = (candDict, calc_score_altogether_in_sentence(comb, candDict, scoreDictDict))
    return combCandDict

In [25]:
def update_comb_cand_dict_score_deprecated(combCandDict, scoreDictDict=scoreDictDict):
    for idx, weightTypeList in combCandDict.items():
        typeNum = len(weightTypeList)
        for i, (comb, candList) in enumerate(weightTypeList):
            localCandScoreSum = sum([candScore for _,candScore in candList])
            #TODO: Validate this
            sentenceTotalScore = len(comb.get_sentence()) - sum(comb.get_weight_dict('dummy').values())
            for j, (candDict, candScore) in enumerate(candList):
                newScoreSum = 0
                for tagsetType, cand in candDict.items():
                    newScoreSum += sentence_tagset_score_with_weight(comb.get_sentence(), \
                                                                     cand, comb.get_weight_dict(tagsetType), scoreDictDict, \
                                                                     maxScoreDict)
                candList[j] = (candDict, newScoreSum)
    return combCandDict

In [26]:
def calc_building_score(combCandDict):
    buildingScore = 0
    for idx, weightTypeList in combCandDict.items():
        rmvIdxList = list()
        for i, candList in enumerate(weightTypeList):
            if len(candList[1])==0:
                rmvIdxList.append(i)
        rmvIdxList.reverse()
        for i in rmvIdxList:
            del weightTypeList[i]
        typeNum = len(weightTypeList)
        for (comb, candList) in weightTypeList:
            localCandScoreSum = sum([score for _, score in candList])
            for cand, candScore in candList:
                buildingScore += candScore * candScore / localCandScoreSum / typeNum
        #print idx, buildingScore
    return buildingScore

buildingScoreList = [calc_building_score(origCombCandDict)]
print buildingScoreList

[517.92403620730249]


In [27]:
def eval_word_meaning((combCandDict, word, scoreDictDict, baseBuildingScore)):
    #print "comb", type(combCandDict)
    #print "word", type(word), word
    #print "score", type(scoreDictDict)
    scoreDict = scoreDictDict[word]
    if len(scoreDict)<=1:
        return None
    baseCombCandDict = deepcopy(combCandDict)
    removeCandList = scoreDict.keys()
    updatedCombCandDictList = list()
    for removeCand in removeCandList:
        updatedScoreDictDict = deepcopy(scoreDictDict)
        del updatedScoreDictDict[word][removeCand]
        combCandDict = deepcopy(baseCombCandDict)
        updatedCombCandDictList.append(update_comb_cand_dict_score(combCandDict, scoreDictDict=updatedScoreDictDict))
    updatedScoreList = list()
    for updatedCombCandDict in updatedCombCandDictList:
        updatedScoreList.append(calc_building_score(updatedCombCandDict))
    if max(updatedScoreList) > baseBuildingScore:
        selectedScore = max([score for score in updatedScoreList if [score - baseBuildingScore]>0])
        removeTagset = removeCandList[updatedScoreList.index(selectedScore)]
        return (word, removeTagset)
    else:
        return None
    
def eval_word_meaning2((combCandDict, word, scoreDictDict, baseBuildingScore)):
    #print "comb", type(combCandDict)
    #print "word", type(word), word
    #print "score", type(scoreDictDict)
    scoreDict = scoreDictDict[word]
    if len(scoreDict)<=1:
        return None
    #baseCombCandDict = deepcopy(combCandDict)
    baseCombCandDict = dict()
    for i, combCand in combCandDict.items():
        if word in combCand[0][0].get_sentence():
            baseCombCandDict[i] = deepcopy(combCand)
    if len(baseCombCandDict)<1:
        return None
    baseBuildingScore = calc_building_score(baseCombCandDict)
    
    removeCandList = scoreDict.keys()
    updatedCombCandDictList = list()
    for removeCand in removeCandList:
        updatedScoreDictDict = deepcopy(scoreDictDict)
        del updatedScoreDictDict[word][removeCand]
        combCandDict = deepcopy(baseCombCandDict)
        updatedCombCandDictList.append(update_comb_cand_dict_score(combCandDict, scoreDictDict=updatedScoreDictDict))
    updatedScoreList = list()
    for updatedCombCandDict in updatedCombCandDictList:
        updatedScoreList.append(calc_building_score(updatedCombCandDict))
    
    baseCombCandDict = None
    combCandDict = None
    updatedScoreDictDict = None
    
    if max(updatedScoreList) > baseBuildingScore:
        selectedScore = max([score for score in updatedScoreList if [score - baseBuildingScore]>0])
        removeTagset = removeCandList[updatedScoreList.index(selectedScore)]
        return (word, removeTagset)
    else:
        return None
    
wordUpdateIterationNum = 0

In [28]:
plus = lambda x,y:x+y
minus = lambda x,y:x-y
alargerthanb = lambda a,b:a>b
asmallerthanb = lambda a,b:a<b
posLimit = 1
negLimit = 0
largerthan1 = lambda x:x>1
smallerthan0 = lambda x:x<0
baseDeltaWeight = 0.1

def try_change_weight(word, tagsetType, candWeightDictDict, weightDict, tagsetTypeTotalNum, baseDeltaWeight=baseDeltaWeight, changeFunc=plus, affectedFunc=minus, \
                      changeLimit=posLimit, affectedLimit=negLimit, changeCriterion=largerthan1, affectedCriterion=smallerthan0):
    candWeightDictDict[tagsetType][word] = changeFunc(weightDict[word], baseDeltaWeight)
    if changeCriterion(candWeightDictDict[tagsetType][word]):
        candWeightDictDict[tagsetType][word] = changeLimit
    deltaWeight = abs(candWeightDictDict[tagsetType][word] - weightDict[word])
    for otherTagsetType, otherWeightDict in candWeightDictDict.items():
        if otherTagsetType==tagsetType or deltaWeight==0:
            continue
        candWeightDictDict[otherTagsetType][word] = affectedFunc(otherWeightDict[word], deltaWeight / (tagsetTypeTotalNum-1))
        if affectedCriterion(candWeightDictDict[otherTagsetType][word]):
            candWeightDictDict[tagsetType][word] = affectedFunc(candWeightDictDict[tagsetType][word],abs(affectedLimit-candWeightDictDict[otherTagsetType][word])) # If there is not enough remaining score in this tagsetType
            candWeightDictDict[otherTagsetType][word] = affectedLimit
    return candWeightDictDict

def eval_weight((combCandDict, scoreDictDict)):
    combCandDict = deepcopy(combCandDict)
    for idx, weightTypeList in combCandDict.items():
        #print 'weight eval', idx
        for (comb, candList) in weightTypeList:
            if len(candList)==0:
                continue
            prevTotalScore = sum([val*val for val in map(itemgetter(1), candList)]) / sum(map(itemgetter(1), candList))
            dummyWeightDict = comb.get_weight_dict('dummy')
            tagsetTypeTotalNum = len(comb.get_total_weight_dict())
            newWeightDictDict = defaultdict(dict)
            for word, _ in dummyWeightDict.items(): #TODO: TODO: NEEDS TO BE SHUFFLED AFTER VERIFICATION
                weightDictDict = comb.get_total_weight_dict()
                candWeightDictDictList = list()
                for tagsetType, weightDict in weightDictDict.items(): 
                    #if tagsetType=='dummy': #TODO: We may need this. Not sure for now
                    #    continue
                    #Positive weight on the word-tagsetType
                    candWeightDictDict = \
                    try_change_weight(word, tagsetType, deepcopy(weightDictDict), weightDict, tagsetTypeTotalNum, baseDeltaWeight=baseDeltaWeight, changeFunc=plus, affectedFunc=minus, \
                      changeLimit=posLimit, affectedLimit=negLimit, changeCriterion=largerthan1, affectedCriterion=smallerthan0)
                    localScoreList = list()
                    for candDict in map(itemgetter(0), candList):
                        #TODO change to the original if it does not help.
                        localScoreList.append(calc_score_altogether_in_sentence(deepcopy(comb).update_total_weight_dict(candWeightDictDict), candDict, scoreDictDict))
                        #localScoreList.append(calc_score_altogether_in_sentence(comb.update_total_weight_dict(candWeightDictDict), candDict, scoreDictDict))
                    score = sum([val*val for val in localScoreList]) / sum(localScoreList)
                    candWeightDictDictList.append((candWeightDictDict, score))
                    #Negative weight on the word-tagsetType
                    candWeightDictDict = \
                    try_change_weight(word, tagsetType, deepcopy(weightDictDict), weightDict, tagsetTypeTotalNum, baseDeltaWeight=baseDeltaWeight, changeFunc=minus, affectedFunc=plus, \
                      changeLimit=negLimit, affectedLimit=posLimit, changeCriterion=smallerthan0, affectedCriterion=largerthan1)
                    localScoreList = list()
                    for candDict in map(itemgetter(0), candList):
                        #try:
                        localScoreList.append(calc_score_altogether_in_sentence(deepcopy(comb).update_total_weight_dict(candWeightDictDict), candDict, scoreDictDict))
                        #localScoreList.append(calc_score_altogether_in_sentence(comb.update_total_weight_dict(candWeightDictDict), candDict, scoreDictDict))
                        #except:
                        #    print candDict, candWeightDictDict
                        #    print weightDictDict
                        #   assert(False)    
                    score = sum([val*val for val in localScoreList]) / sum(localScoreList)
                    candWeightDictDictList.append((candWeightDictDict, score))
                maxScore = max(map(itemgetter(1),candWeightDictDictList))
                if maxScore>prevTotalScore:
                    maxIdx = map(itemgetter(1),candWeightDictDictList).index(maxScore)
                    chosenWeightDictDict = map(itemgetter(0),candWeightDictDictList)[maxIdx]
                else:
                    chosenWeightDictDict = weightDictDict
                for tagsetType, weightDict in chosenWeightDictDict.items():
                    newWeightDictDict[tagsetType][word] = weightDict[word]
            newWeightDictDict = dict(newWeightDictDict)
            comb.update_total_weight_dict(newWeightDictDict)
            for i, (candDict, _) in enumerate(candList):
                newScore = calc_score_altogether_in_sentence(comb, candDict, scoreDictDict)
                candList[i] = (candDict, newScore)
                
    return combCandDict

In [29]:
def remove_weak_candidates(combCandDict):
    for idx, weightTypeList in combCandDict.items():
        #print 'weight eval', idx
        newWeightTypeList = list()
        for i, (comb, candList) in enumerate(weightTypeList):
            
            rmvIdxList = list()
            for j, (candSet, candScore) in enumerate(candList):
                if candScore==0:
                    rmvIdxList.append(j)
            rmvIdxList.reverse()
            for j in rmvIdxList:
                del candList[j]
            if len(candList)>2:
                scoreList = map(itemgetter(1), candList)
                minVal = min(scoreList)
                minIdx = scoreList.index(minVal)
                del candList[minIdx]
                newWeightTypeList.append((comb,candList))
            elif len(candList)==0:
                del weightTypeList[i]
            else:
                newWeightTypeList.append((comb,candList))
        combCandDict[idx] = weightTypeList
    return combCandDict

In [30]:


iterNum = 10



mpFlag=True
#try:
reduceCandNum = 1
for iterCnt in range(0,iterNum):
    print wordUpdateIterationNum
    
    ######################### Init Cand 
    beginTime =  datetime.now()
    if mpFlag:
        pool = multiprocessing.Pool(workerNum)
        #combTupleList = pool.map(init_comb, zip(sentenceList, repeat(subTagListDictList), repeat(scoreDictDict), repeat(reduceCandNum)))
        pool.terminate()
        pool.close()
        pool.join()
        reduceCandNum +=1
    else:
        combTupleList = map(init_comb, zip(sentenceList, repeat(subTagListDictList), repeat(scoreDictDict)))
    #combCandDict = dict((i,combTuple) for i, combTuple in enumerate(combTupleList))
    endTime = datetime.now()
    print "comb init time:", (endTime-beginTime).total_seconds()/60, 'minutes'
    
    
    #print iterCnt
    ########################## Update Weights
    beginTime =  datetime.now()
    baseBuildingScore = buildingScoreList[-1]
    beginTime =  datetime.now()
    if mpFlag:
        pool = multiprocessing.Pool(workerNum)
        makeDict = lambda x:dict([x])
        updatedDictList = pool.map(eval_weight, zip(map(makeDict,combCandDict.items()), repeat(scoreDictDict)))
        dictgetter = lambda x:x.items()
        adder = lambda x,y:x+y
        combCandDict = dict(reduce(adder,map(dictgetter, updatedDictList)))
        pool.terminate()
        pool.close()
        pool.join()
    else:
        combCandDict = eval_weight((combCandDict, scoreDictDict))
    endTime = datetime.now()
    print "weight computing time:", (endTime-beginTime).total_seconds()/60, '(min)'


    #################### Update Word Meaning
    baseBuildingScore = buildingScoreList[-1]
    beginTime =  datetime.now()
    if mpFlag:
        pool = multiprocessing.Pool(workerNum)
        removeWordTagsetTupleList = pool.map(eval_word_meaning2, zip(repeat(combCandDict), scoreDictDict.keys(), \
                                                                     repeat(scoreDictDict), repeat(baseBuildingScore)))
        pool.terminate()
        pool.close()
        pool.join()
    else:
        removeWordTagsetTupleList = map(eval_word_meaning2, zip(repeat(combCandDict), wordList, repeat(scoreDictDict), \
                                                                repeat(baseBuildingScore)))
    removeWordTagsetDict = dict([el for el in removeWordTagsetTupleList if el])
    endTime = datetime.now()

    #update removing word meanings
    for word, tagset in removeWordTagsetDict.items():
        del scoreDictDict[word][tagset]
    combCandDict = update_comb_cand_dict_score(combCandDict, scoreDictDict)
    buildingScoreList.append(calc_building_score(combCandDict))
    wordUpdateIterationNum += 1
    

    ################## Remove weak candidates
    combCandDict = remove_weak_candidates(combCandDict)
    combCandDict = remove_weak_candidates(combCandDict)


    print "word meaning computing time:", (endTime-beginTime).total_seconds()/60, '(min)'
    print buildingScoreList
    print "================"
    
bestCandScoreDict = dict()
for i, combCand in combCandDict.items():
    bestCand = None
    bestScore = 0
    bestComb = None
    for comb, candSetList in combCand:
        for candSet, score in candSetList:
            if score>bestScore:
                bestScore = score
                bestCand = candSet
                bestComb = comb
    bestCandScoreDict[i] = (bestComb, bestCand, bestScore)

0
comb init time: 0.00602315 minutes
weight computing time: 14.8874763167 (min)
word meaning computing time: 3.18745931667 (min)
[517.92403620730249, 589.00232751024498]
1
comb init time: 0.00647025 minutes
weight computing time: 11.8402247833 (min)
word meaning computing time: 2.2997429 (min)
[517.92403620730249, 589.00232751024498, 726.96501303161835]
2
comb init time: 0.00656676666667 minutes
weight computing time: 9.61724175 (min)
word meaning computing time: 1.67433085 (min)
[517.92403620730249, 589.00232751024498, 726.96501303161835, 849.88124206055625]
3
comb init time: 0.00667526666667 minutes
weight computing time: 7.7905358 (min)
word meaning computing time: 1.25544383333 (min)
[517.92403620730249, 589.00232751024498, 726.96501303161835, 849.88124206055625, 991.2747872734534]
4
comb init time: 0.00750061666667 minutes
weight computing time: 6.23813021667 (min)
word meaning computing time: 0.956170633333 (min)
[517.92403620730249, 589.00232751024498, 726.96501303161835, 849.88

In [31]:
pointIdxDict = defaultdict(list)
for i, (bestComb, bestCand, bestScore) in bestCandScoreDict.items():
    pointIdxDict[bestCand['point']].append(i)
avgScorePerPointDict = dict()
for key, idxList in pointIdxDict.items():
    avgScorePerPointDict[key] = np.mean([cand[2] for cand in [bestCandScoreDict[i] for i in idxList]])
for key, mean in sorted(avgScorePerPointDict.items(), key=itemgetter(1)):
    #print key, mean, len(pointIdxDict[key])
    pass
    
lenGetter = lambda x: len(x[1])
totalScoreGetter = lambda (key,avg):avg*len(pointIdxDict[key])

weakestPointList = list()
#for key, idxList in sorted(pointIdxDict.items(), key=lenGetter, reverse=True):
#    print key, len(pointIdxDict[key])
for key, avg in sorted(avgScorePerPointDict.items(), key=totalScoreGetter):
    num = len(pointIdxDict[key])
    if num>3:
        print key, avg, num, num*avg
        weakestPointList.append(key)

proportional_gain_setpoint 0.184103634527 4 0.736414538109
exhaust_fan_status 0.238188887025 4 0.952755548102
humidifier_panel_no_water_alarm 0.265206092863 4 1.06082437145
humidify_command 0.268370953952 4 1.07348381581
stages_status 0.272300761631 4 1.08920304653
vfd_speed_sensor 0.282983535048 4 1.13193414019
cooling_on_off_status 0.300165674131 4 1.20066269653
manual_auto_status 0.30510630226 4 1.22042520904
occupied_cooling_max_discharge_air_flow_setpoint 0.255310442451 5 1.27655221225
exhaust_fan_off_status_led 0.257109271689 5 1.28554635845
cooling_valve_command 0.331522721679 4 1.32609088671
exhaust_fan_command 0.270023687383 5 1.35011843692
emergency_power_off_activated_by_high_temperature_status 0.337563668031 4 1.35025467212
supply_fan_start_stop_status 0.343496386667 4 1.37398554667
run_direction_status 0.34946680959 4 1.39786723836
air_flow_sensor 0.175694589538 8 1.40555671631
change_filter_alarm 0.361121085951 4 1.44448434381
heating_on_off_status 0.377851967345 4 1.5114

In [32]:
#### Asking words' meaning to experts

def pick_weak_word_in_cluster(idxList):
    adder = lambda x,y:x+y
    wordOccurrenceDict = dict()
    wordUsageDict = dict()
    occIdxDict = dict()
    cnt = 0
    wordEntropyDict = dict()
    for word, scoreDict in scoreDictDict.items():
        wordEntropyDict[word] = stats.entropy(scoreDict.values())
        cnt += 1
        if cnt%100==0:
            print cnt
        occIndices = [idx for idx, sentence in zip(idxList, [sentenceList[i] for i in idxList]) if word in sentence]
        occIdxDict[word] = occIndices
        wordOccurrence = len(occIndices)
        wordOccurrenceDict[word] = wordOccurrence
        if wordOccurrence==0 or len(scoreDict)==1:
            continue
        usageScore = 0
        for idx in occIndices:
            combDictList = combCandDict[idx]
            candSetList = reduce(adder, map(itemgetter(1), combDictList), [])
            totalScore = sum(map(itemgetter(1), candSetList))
            subScore = 0
            wordCandScoreSum = sum(scoreDict.values())
            for wordTagsetCand, tagsetScore in scoreDict.items():
                wordTagsetCand = wordTagsetCand.split()
                for tagCand in wordTagsetCand:
                    for candSet, candScore in candSetList:
                        for cand in candSet.values():
                            cand = cand.split('_')
                            if tagCand in cand:
                                subScore += tagsetScore/wordCandScoreSum/len(wordTagsetCand) * candScore / totalScore / len(cand)
            usageScore += subScore
        wordUsageDict[word] = usageScore

    percentUsageDict = dict([(word, wordUsageDict[word]/wordOccurrenceDict[word]) for word in wordUsageDict.keys()])

    sortedUsage = sorted(percentUsageDict.values())
    print "========"
    for word, usage in sorted(percentUsageDict.items(), key=itemgetter(1)):
        if wordOccurrenceDict[word]>2 and len(word)>1:
            print word, usage, wordOccurrenceDict[word], wordEntropyDict[word]
    print "========"
    return percentUsageDict

percentUsageDict = pick_weak_word_in_cluster(range(0,coverageNum))
            
for pointTagset in weakestPointList[0:5]:
    pick_weak_word_in_cluster(pointIdxDict[pointTagset])

100
200
th 0 7 1.97205927607
adj 1.35149761578e-05 238 0.953999447413
programming 0.00676569649596 68 1.95866255588
rd 0.00751409943863 7 2.5475282291
actual 0.0117655009149 24 1.24093932303
htgpid 0.0129142721142 220 2.62918232671
modules 0.0214606944303 21 2.17143398828
stpt 0.047178923429 20 1.35732212463
do 0.0613381310771 24 0.622230586748
acthtgsp 0.0727255927782 220 1.25699519811
reference 0.0800127083063 24 1.03599093395
osa 0.0833561258566 7 0.693144973473
rf 0.0850408908683 104 1.55778405008
ovrd 0.0884999994277 4 0.637897771749
chwp 0.0891705534954 121 0.655766097306
ef 0.0921794934647 10 1.24636459306
oa 0.093712988358 3 1.580712474
clg 0.0990632959302 27 1.56874765136
sts 0.119363585571 33 1.06101238146
ref 0.127392341144 26 0.642744231932
hitemp 0.150353056332 5 0.550297246512
glycool 0.150533112723 4 0.0
cmd 0.151773017975 288 0.610864302055
clgmaxflo 0.154939455177 236 0.949340514052
clgminflo 0.158018603998 235 1.06128857722
dx 0.16169735963 139 0.571259473646
dir 0.17

In [33]:
commonWordListDict = defaultdict(set)
adder = lambda x,y:x+y
for pointTagset, idxList in pointIdxDict.items():
    if pointTagset!='room_temperature_sensor':
        pass
    subSentenceList = [sentenceList[i] for i in idxList]
    aSentence = list(set(reduce(adder, subSentenceList,[])))
    for word in aSentence:
        cnt = 0
        for sentence in subSentenceList:
            if word in sentence:
                cnt += 1
            if cnt/float(len(idxList)) > 0.25:
                commonWordListDict[pointTagset].add(word)

In [34]:
shownWordList = list()
anchorListDict = defaultdict(list)
for pointTagset, commonWordList in commonWordListDict.items():
    for word in commonWordList:
        if word not in shownWordList:
            if word in percentUsageDict.keys() and len(word)>1:
                print word, percentUsageDict[word]
                shownWordList.append(word)
                anchorListDict[pointTagset].append((word, percentUsageDict[word]))

boxmode 0.33918891503
cmd 0.151773017975
sf 0.240715150087
hw 0.346858445335
dx 0.16169735963
vlv 0.360293191218
actual 0.0117655009149
ai 0.213199859665
remt 0.224599311962
bsbm 0.443665833028
chw 0.276083978592
programming 0.00676569649596
pager 0.0395833369482
hihum 0.166215036826
stpt 0.047178923429
chwr 0.188156799353
chwp 0.0891705534954
hwp 0.179353289184
epo 0.177559960015
alrmpres 0.324077913545
compres 0.309482507694
shutdn 0.178423284561
dec 0.223930771394
batt nan
osa 0.0833561258566
reference 0.0800127083063
rf 0.0850408908683
ref 0.127392341144
fcwsp 0.214082484386
fcwdp 0.00992914111763
do 0.0613381310771
dmpr 0.315024722333
rdy 0.317016513225
dir 0.172628597766
pid 0.0243924320654
clgminflo 0.158018603998
loc 0
sts 0.119363585571
fcw 0
clg 0.0990632959302
occhtgfl 0.152383587313
ef 0.0921794934647
humd 0.312576684119
tempsetf 0.271740646637
freq 0.335841816816
zn 0.291874915212
mtw 0.0525209708273
mtr 0.193842823286
fltr 0.459048866216
tempalm 0.14608694359
hihitempalm 

In [35]:
scoreDictDict

{'a': {u'ahu': 2.7320378151260503,
  u'air': 2.7320378151260503,
  u'auto': 2.6873949579831935,
  u'band': 0.9642857142857144,
  u'fan': 1.9444444444444442,
  u'lag': 1.9444444444444442,
  u'max': 1.9444444444444442,
  u'warm': 0.9642857142857144},
 'acc': {u'acceleration': 3.135135135135135,
  u'acceleration time setpoint': 2.0416262416262416},
 'aclg': {u'angle': 1.0,
  u'auto cooling temperature setpoint': 2.0,
  u'average cooling demand sensor': 0.9810199942045783,
  u'close': 0.9178580438987468,
  u'crac': 1.0,
  u'cycle': 1.0,
  u'discharge air temperature cooling setpoint': 0.828631452581033,
  u'fault': 1.0,
  u'hot water coil entering temperature sensor': 0.9484616892024301,
  u'hvac': 1.0,
  u'lag': 0.9807956104252401,
  u'space': 0.9178580438987468,
  u'trace': 0.9178580438987468},
 'act': {u'city': 0.8666666666666666},
 'actclgsp': {u'space': 1.0},
 'acthtgsp': {u'activated': 2.5940358259433323,
  u'discharge air temperature': 1.0,
  u'trace heat sensor': 0.784313725490196,

In [36]:

itemNumGetter = lambda x:len(pointIdxDict[x[0]])

for pointTagset, anchorList in sorted(anchorListDict.items(), key=itemNumGetter, reverse=True):
    print pointTagset
    print '\t', sorted(anchorList, key=itemgetter(1))

room_temperature_setpoint
	[('adj', 1.3514976157835226e-05), ('commonsp', 0.044770037022228051)]
room_temperature_sensor
	[('acthtgsp', 0.072725592778192044)]
heat_request_percent_setpoint
	[('htgpid', 0.012914272114194561), ('dpr', 0.058280250576924438)]
zone_temperature_sensor
	[('zn', 0.29187491521225739)]
damper_position_sensor
	[('dmpr', 0.31502472233318557)]
cooling_max_supply_air_flow_setpoint
	[('clgmaxflo', 0.15493945517721888)]
supply_air_flow_sensor
	[('sup', 0.23890363017023683)]
cooling_min_discharge_air_flow_setpoint
	[('clgminflo', 0.15801860399795867)]
reheat_valve_command
	[('rh', 0.29126598567047202)]
supply_fan_vfd_speed_command
	[('cmd', 0.15177301797474155), ('sf', 0.24071515008737804)]
return_fan_vfd_speed_command
	[('reference', 0.080012708306273592), ('rf', 0.085040890868307281), ('ref', 0.12739234114442816)]
average_cooling_demand_sensor
	[('th', 0), ('rd', 0.0075140994386323949)]
supply_fan_vfd_speed_sensor
	[('actual', 0.011765500914902423), ('ai', 0.21319985

In [37]:
def change_word_meaning(word, meaning, scoreDictDict=scoreDictDict, maxScoreDict=maxScoreDict):
    scoreDictDict[word] = {meaning: maxScoreDict[word]}
    return scoreDictDict

In [38]:
'''
change_word_meaning('actclgsp', 'effective cooling setpoint')
change_word_meaning('acthtgsp', 'effective heating setpoint')
change_word_meaning('dpr', 'damper')
change_word_meaning('commonsp', 'temperature setpoint')
change_word_meaning('clgpid', 'cooling command')
change_word_meaning('pos', 'position')
change_word_meaning('rh', 'reheat')
change_word_meaning('reference', 'xxxxxxxxxxx')
change_word_meaning('stpt', 'setpoint')
change_word_meaning('chwp', 'chilled water pump')
change_word_meaning('rf', 'return fan')
change_word_meaning('htgpid', 'heating command')
change_word_meaning('actual', 'xxxxxxxxxxxxxx')
change_word_meaning('ai', 'xxxxxxxxxxxxxxxxxx')
change_word_meaning('ctrl', 'xxxxxxxxxxxxxxx')
change_word_meaning('act', 'xxxxxxxxxxxxxxxxx')
change_word_meaning('cmd', 'command')
change_word_meaning('ao', 'xxxxxxxxxxxxx')
change_word_meaning('ref', 'reference')
change_word_meaning('hwp', 'hot water pump')
change_word_meaning('vnd', 'xxxxxxxxxxxxxxx')
'''




'''


change_word_meaning('sys',  'system')
change_word_meaning('sup', 'supply')
change_word_meaning('pos', 'position')
change_word_meaning('chwp', 'chilled water pump')
change_word_meaning('dp', 'differential pressure')
change_word_meaning('actual', 'xxxxxxxxxxxxxxxxxxx')
change_word_meaning('ctrl', 'xxxxxxxxxxxxxxxx')
change_word_meaning('act', 'xxxxxxxxxxxx')
change_word_meaning('chw', 'chilled water')
change_word_meaning('mtws', 'water supply temperature')
change_word_meaning('actclgsp', 'effective cooling setpoint')
change_word_meaning('acthtgsp', 'effective heating setpoint')

change_word_meaning('pid', 'xxxxxxxxxxxxxxxx')
change_word_meaning('pct', 'xxxxxxxxxxxxxxxxx')
change_word_meaning('hwp', 'hot water pump')
change_word_meaning('vnd', 'xxxxxxxxxxxxxx')
change_word_meaning('ai', 'xxxxxxxxxxxxxxxx')
change_word_meaning('sf', 'supply fan')
'''

"\n\n\nchange_word_meaning('sys',  'system')\nchange_word_meaning('sup', 'supply')\nchange_word_meaning('pos', 'position')\nchange_word_meaning('chwp', 'chilled water pump')\nchange_word_meaning('dp', 'differential pressure')\nchange_word_meaning('actual', 'xxxxxxxxxxxxxxxxxxx')\nchange_word_meaning('ctrl', 'xxxxxxxxxxxxxxxx')\nchange_word_meaning('act', 'xxxxxxxxxxxx')\nchange_word_meaning('chw', 'chilled water')\nchange_word_meaning('mtws', 'water supply temperature')\nchange_word_meaning('actclgsp', 'effective cooling setpoint')\nchange_word_meaning('acthtgsp', 'effective heating setpoint')\n\nchange_word_meaning('pid', 'xxxxxxxxxxxxxxxx')\nchange_word_meaning('pct', 'xxxxxxxxxxxxxxxxx')\nchange_word_meaning('hwp', 'hot water pump')\nchange_word_meaning('vnd', 'xxxxxxxxxxxxxx')\nchange_word_meaning('ai', 'xxxxxxxxxxxxxxxx')\nchange_word_meaning('sf', 'supply fan')\n"

In [39]:
#### Asking words' meaning to experts
adder = lambda x,y:x+y
wordOccurrenceDict = dict()
wordUsageDict = dict()
occIdxDict = dict()
cnt = 0
wordEntropyDict = dict()
for word, scoreDict in scoreDictDict.items():
    wordEntropyDict[word] = stats.entropy(scoreDict.values())
    cnt += 1
    if cnt%100==0:
        print cnt
    occIndices = [i for i, sentence in enumerate(sentenceList[0:coverageNum]) if word in sentence]
    occIdxDict[word] = occIndices
    wordOccurrence = len(occIndices)
    wordOccurrenceDict[word] = wordOccurrence
    if wordOccurrence==0 or len(scoreDict)==1:
        continue
    usageScore = 0
    for idx in occIndices:
        combDictList = combCandDict[idx]
        candSetList = reduce(adder, map(itemgetter(1), combDictList), [])
        totalScore = sum(map(itemgetter(1), candSetList))
        subScore = 0
        wordCandScoreSum = sum(scoreDict.values())
        for wordTagsetCand, tagsetScore in scoreDict.items():
            wordTagsetCand = wordTagsetCand.split()
            for tagCand in wordTagsetCand:
                for candSet, candScore in candSetList:
                    for cand in candSet.values():
                        cand = cand.split('_')
                        if tagCand in cand:
                            subScore += tagsetScore/wordCandScoreSum/len(wordTagsetCand) * candScore / totalScore / len(cand)
        usageScore += subScore
    wordUsageDict[word] = usageScore

percentUsageDict = dict([(word, wordUsageDict[word]/wordOccurrenceDict[word]) for word in wordUsageDict.keys()])

sortedUsage = sorted(percentUsageDict.values())
for word, usage in sorted(percentUsageDict.items(), key=itemgetter(1)):
    if wordOccurrenceDict[word]>10 and len(word)>1:
        print word, usage, wordOccurrenceDict[word], wordEntropyDict[word]

100
200
adj 1.35149761578e-05 238 0.953999447413
programming 0.00676569649596 68 1.95866255588
actual 0.0117655009149 24 1.24093932303
htgpid 0.0129142721142 220 2.62918232671
modules 0.0214606944303 21 2.17143398828
stpt 0.047178923429 20 1.35732212463
do 0.0613381310771 24 0.622230586748
acthtgsp 0.0727255927782 220 1.25699519811
reference 0.0800127083063 24 1.03599093395
rf 0.0850408908683 104 1.55778405008
chwp 0.0891705534954 121 0.655766097306
clg 0.0990632959302 27 1.56874765136
sts 0.119363585571 33 1.06101238146
ref 0.127392341144 26 0.642744231932
cmd 0.151773017975 288 0.610864302055
clgmaxflo 0.154939455177 236 0.949340514052
clgminflo 0.158018603998 235 1.06128857722
dx 0.16169735963 139 0.571259473646
htgflow 0.192659904222 220 1.33958814098
flt 0.225480232153 16 0.688359541883
sup 0.23890363017 235 0.678937716196
supflo 0.248519565281 241 0.686961576597
rh 0.29126598567 225 0.692461152611
zn 0.291874915212 239 1.039661933
humd 0.312576684119 20 0.586845195498
freq 0.3358

In [40]:
for word, occ in sorted(wordOccurrenceDict.items(), key=itemgetter(1), reverse=True):
    print word, occ

nae 4577
ebu 4562
b 4551
n 4499
rm 3880
vma 3852
command 2835
setpoint 2458
sensor 1371
meter 1360
percent 1287
temperature 1231
flow 1184
t 568
c 528
vnd 508
o 461
vfd 310
occ 302
status 302
alarm 302
cmd 288
sp 260
supflo 241
zn 239
adj 238
w 238
commonsp 237
clgmaxflo 236
actclgsp 236
dpr 236
clgpid 236
sup 235
clgminflo 235
dmpr 234
pos 234
vlv 230
rh 225
htgpid 220
acthtgsp 220
htgflow 220
crac 200
a 184
dx 139
chwp 121
sf 105
rf 104
ah 74
programming 68
pid 68
hwp 51
sys 42
chw 36
sts 33
avg 33
average 33
cooling 32
server 32
current 31
fan 30
demand 30
hw 29
file 29
xxx 28
floor 28
clg 27
ref 26
tuning 25
hi 25
do 24
reference 24
actual 24
ai 24
flr 24
fschw 22
system 22
humidity 21
modules 21
programmable 21
points 21
humd 20
stpt 20
reset 20
hx 19
temp 19
run 18
pm 17
direction 17
d 16
flt 16
co 16
f 16
pan 16
ss 16
lo 14
fault 14
frequency 14
freq 14
torque 14
enable 13
spd 13
pct 12
loss 12
output 12
direct 12
act 12
unit 12
ao 12
curr 12
ctrl 12
lock 12
lim 12
s 12
gain 12


In [41]:
def check_correct(predLabel, currLabel):
    currLabel = currLabel.replace(' ', '_')
    possibleLabelList = [currLabel]
    if 'supply' in currLabel:
        possibleLabelList.append(currLabel.replace('supply', 'discharge'))
    if 'discharge' in currLabel:
        possibleLabelList.append(currLabel.replace('discharge', 'supply'))
    if currLabel in equalDict.keys():
        possibleLabelList.append(equalDict[currLabel])
    if predLabel in possibleLabelList:
        return True
    else:
        return False

In [42]:
filename = 'result/'+buildingName+'_comb_cand.csv'
with open(filename, 'wb') as fp:
    writer = csv.writer(fp)
    writer.writerow(['sentence', 'true point type', 'true equip type', 'point correct?', 'candidate combi', 'score', 'candidate combi', 'score'])
    addFunc = lambda x,y:x+y

    for i, tagsetTypeBasedList in combCandDict.items():
        candTupleList = reduce(addFunc,map(itemgetter(1), tagsetTypeBasedList),[])
        candTupleList = sorted(candTupleList, key=itemgetter(1), reverse=True)
        comb = tagsetTypeBasedList[0][0]
        srcid = sensorDF.index[i]
        trueRow = trueDF.loc[srcid]
        pointCorrect = check_correct(bestCandScoreDict[i][1]['point'], trueRow['Schema Label'])
        writeList = [comb.get_sentence(), trueRow['Schema Label'], trueRow['Equipment Type'], 1 if pointCorrect else 0]
        
        for candTuple in candTupleList:
            writeList.append(candTuple[0])
            writeList.append(candTuple[1])
        writer.writerow(writeList)
f = gdrive.CreateFile()
f.SetContentFile(filename)
f.Upload()
f = None

In [43]:
print "Finished!!!"
from IPython.display import Audio
sound_file = 'etc/fins_success.wav'
Audio(url=sound_file, autoplay=True)

Finished!!!


In [44]:
# select best cand set.
labelSentenceList = list()
selectedTagsetDict = dict()
for i, candComb in combCandDict.items():
    dummyWeightDict = bestCandScoreDict[i][0].get_weight_dict('dummy')
    labelWordList = list()
    for word, weight in dummyWeightDict.items():
        if weight>0.8:
            labelWordList.append(word)
    for word in rawSentenceList[i]:
        if word.isdigit():
            labelWordList.append(word)
    labelSentenceList.append(labelWordList)

In [45]:
for i in range(0,len(combCandDict)):
    if 'act' in combCandDict[i][0][0].get_sentence():
        print i, combCandDict[i][0][0].get_sentence()

53 ['nae', 'nae', 'vnd', 'chwp', 'vfd', 'pct', 'act', 'ebu', 'chwp', 'pct', 'act', 'percent', 'sensor', 'meter']
60 ['nae', 'nae', 'vnd', 'chwp', 'vfd', 'pct', 'act', 'ebu', 'chwp', 'pct', 'act', 'percent', 'sensor', 'meter']
67 ['nae', 'nae', 'vnd', 'hwp', 'vfd', 'pct', 'act', 'ebu', 'hwp', 'pct', 'act', 'percent', 'sensor', 'meter']
74 ['nae', 'nae', 'vnd', 'hwp', 'vfd', 'pct', 'act', 'ebu', 'hwp', 'pct', 'act', 'percent', 'sensor', 'meter']
81 ['nae', 'nae', 'vnd', 'sf', 'vfd', 'pct', 'act', 'ebu', 'sf', 'pct', 'act', 'percent', 'sensor', 'meter']
88 ['nae', 'nae', 'vnd', 'sf', 'vfd', 'pct', 'act', 'ebu', 'sf', 'pct', 'act', 'percent', 'sensor', 'meter']
95 ['nae', 'nae', 'vnd', 'sf', 'vfd', 'pct', 'act', 'ebu', 'sf', 'pct', 'act', 'percent', 'sensor', 'meter']
102 ['nae', 'nae', 'vnd', 'sf', 'vfd', 'pct', 'act', 'ebu', 'sf', 'pct', 'act', 'percent', 'sensor', 'meter']
109 ['nae', 'nae', 'vnd', 'rf', 'vfd', 'pct', 'act', 'ebu', 'rf', 'pct', 'act', 'percent', 'sensor', 'meter']
116 [

In [46]:
comb = deepcopy(combCandDict[497][1][0])
comb.weightDictHistory[-1] = {'dummy': {'crac': 0,
  'ebu': 1.0,
  'humidity': 0,
  'lohum': 0,
  'meter': 0,
  'nae': 1.0,
  'sensor': 0,
  'stpt': 1,
  'vnd': 1},
 'equip': {'crac': 1,
  'ebu': 0.0,
  'humidity': 0,
  'lohum': 0,
  'meter': 0,
  'nae': 0.0,
  'sensor': 0,
  'stpt': 0,
  'vnd': 0},
 'point': {'crac': 0,
  'ebu': 0.0,
  'humidity': 1,
  'lohum': 1,
  'meter': 0,
  'nae': 0.0,
  'sensor': 0,
  'stpt': 0,
  'vnd': 0}}


print calc_score_altogether_in_sentence(comb, {'equip': 'crac', 'point':'low_humidity_alarm_setpoint'}, scoreDictDict)

0.334884983556


In [47]:
origScoreDictDict['lohum']

{u'low humidity alarm': 3.0}

In [48]:
comb = deepcopy(combCandDict[497][1][0])
comb.weightDictHistory[-1] = {'dummy': {'crac': 0,
  'ebu': 1.0,
  'humidity': 0,
  'lohum': 0,
  'meter': 0,
  'nae': 1.0,
  'sensor': 0,
  'stpt': 1,
  'vnd': 0},
 'equip': {'crac': 1,
  'ebu': 0.0,
  'humidity': 0,
  'lohum': 0,
  'meter': 0,
  'nae': 0.0,
  'sensor': 0,
  'stpt': 0,
  'vnd': 0},
 'point': {'crac': 0,
  'ebu': 0.0,
  'humidity': 1,
  'lohum': 1,
  'meter': 0,
  'nae': 0.0,
  'sensor': 1,
  'stpt': 1,
  'vnd': 0}}


print calc_score_altogether_in_sentence(comb, {'equip': 'crac', 'point':'low humidity alarm setpoint'}, scoreDictDict)

0.0909090909091


In [49]:
combCandDict[497][1][0].get_total_weight_dict()

{'dummy': {'crac': 0.0031250000000001277,
  'ebu': 1.0,
  'humidity': 0.0031250000000001277,
  'lohum': 0.0062500000000002,
  'meter': 0.49999999999999994,
  'nae': 1.0,
  'sensor': 0.0031250000000001277,
  'stpt': 0.49999999999999994,
  'vnd': 0.49999999999999994},
 'equip': {'crac': 0.996875,
  'ebu': 0.0,
  'humidity': 0,
  'lohum': 0,
  'meter': 0.5,
  'nae': 0.0,
  'sensor': 0,
  'stpt': 0.5,
  'vnd': 0.5},
 'point': {'crac': 0,
  'ebu': 0.0,
  'humidity': 0.996875,
  'lohum': 0.99375,
  'meter': 0,
  'nae': 0.0,
  'sensor': 0.996875,
  'stpt': 0,
  'vnd': 0}}

In [50]:
origScoreDictDict['da']

{u'dead': 2.0}

In [51]:
for word, occ in sorted(wordOccurrenceDict.items(), key=itemgetter(1), reverse=True):
    print word, occ

nae 4577
ebu 4562
b 4551
n 4499
rm 3880
vma 3852
command 2835
setpoint 2458
sensor 1371
meter 1360
percent 1287
temperature 1231
flow 1184
t 568
c 528
vnd 508
o 461
vfd 310
occ 302
status 302
alarm 302
cmd 288
sp 260
supflo 241
zn 239
adj 238
w 238
commonsp 237
clgmaxflo 236
actclgsp 236
dpr 236
clgpid 236
sup 235
clgminflo 235
dmpr 234
pos 234
vlv 230
rh 225
htgpid 220
acthtgsp 220
htgflow 220
crac 200
a 184
dx 139
chwp 121
sf 105
rf 104
ah 74
programming 68
pid 68
hwp 51
sys 42
chw 36
sts 33
avg 33
average 33
cooling 32
server 32
current 31
fan 30
demand 30
hw 29
file 29
xxx 28
floor 28
clg 27
ref 26
tuning 25
hi 25
do 24
reference 24
actual 24
ai 24
flr 24
fschw 22
system 22
humidity 21
modules 21
programmable 21
points 21
humd 20
stpt 20
reset 20
hx 19
temp 19
run 18
pm 17
direction 17
d 16
flt 16
co 16
f 16
pan 16
ss 16
lo 14
fault 14
frequency 14
freq 14
torque 14
enable 13
spd 13
pct 12
loss 12
output 12
direct 12
act 12
unit 12
ao 12
curr 12
ctrl 12
lock 12
lim 12
s 12
gain 12
